![](https://www.qads.com.br/img/QADS.png)

#  Exercícios do módulo de Obtenção e Preparação de Dados

Remis Balaniuk, PhD

# Flat files

Obtenha os dados abertos das [viagens realizadas a serviço no Portal daTransparência](!http://www.portaltransparencia.gov.br/download-de-dados/viagens), processe as colunas normalizando atributos textuais e convertendo os demais para os tipos corretos e ao final salve o(s) dataframe(s) em disco usando o Pickle.

Download do csv do portal da transparência

In [1]:
import requests 
import zipfile
import os

from google.colab import drive

def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

drive.mount('/content/drive')

%cd /content/drive/MyDrive/'Academia QADS'/'Obtendo e Preparando dados'/


if not os.path.exists("transparencia.zip"):
  download_url("http://www.portaltransparencia.gov.br/download-de-dados/viagens/2021", 'transparencia.zip' )

  with zipfile.ZipFile("transparencia.zip", 'r') as zip_ref:
      zip_ref.extract("2021_Viagem.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Academia QADS/Obtendo e Preparando dados


Leitura do arquivo de viagens e tratamento do nome das colunas

In [2]:
import pandas as pd
import unicodedata
import re
from typing import List

df = pd.read_csv("2021_Viagem.csv", sep=";", encoding="ISO8859-1")

def convert_to_snake_case_variable(input : List[str] ) -> List[str]:
  output = []
  for item in input: 
    item = re.sub(r'[\(\)\-]', "", item)
    item = re.sub(r'\s+', "_", item)
    item = unicodedata.normalize("NFKD", item.lower()).encode('ascii', errors='ignore').decode('utf-8')
    output.append(item)
  return output

df.columns = pd.Index(convert_to_snake_case_variable(df.columns.to_list()))

Exemplo dos dados a partir do topo do dataframe

In [3]:
df.head(1)

,identificador_do_processo_de_viagem,numero_da_proposta_pcdp,situacao,viagem_urgente,justificativa_urgencia_viagem,codigo_do_orgao_superior,nome_do_orgao_superior,codigo_orgao_solicitante,nome_orgao_solicitante,cpf_viajante,nome,cargo,funcao,descricao_funcao,periodo_data_de_inicio,periodo_data_de_fim,destinos,motivo,valor_diarias,valor_passagens,valor_outros_gastos
0,16829271,000163/21,Realizada,SIM,A viagem estva programada para ser realizda em...,36000,Ministério da Saúde,36211,Fundação Nacional de Saúde,***.980.641-**,VILIDIANA MORAES MOURA,ENGENHEIRO,FCT-03,Função Comissionada Técnica,03/02/2021,04/02/2021,Nova Brasilândia/MT,Visita técnica no municipio de Nova Brasilãndi...,"223,86","0,00","0,00"


Tipos de dados do dataframe

In [4]:
df.dtypes

identificador_do_processo_de_viagem     int64
numero_da_proposta_pcdp                object
situacao                               object
viagem_urgente                         object
justificativa_urgencia_viagem          object
codigo_do_orgao_superior                int64
nome_do_orgao_superior                 object
codigo_orgao_solicitante                int64
nome_orgao_solicitante                 object
cpf_viajante                           object
nome                                   object
cargo                                  object
funcao                                 object
descricao_funcao                       object
periodo_data_de_inicio                 object
periodo_data_de_fim                    object
destinos                               object
motivo                                 object
valor_diarias                          object
valor_passagens                        object
valor_outros_gastos                    object
dtype: object

Funções para auxiliar na verificação dos tipos de dados

In [5]:
def category(series):
    unique_count = series.nunique(dropna=False)
    total_count = len(series)
    if pd.api.types.is_numeric_dtype(series):
        return 'Numérico'
    elif pd.api.types.is_datetime64_dtype(series):
        return 'Data'
    elif unique_count==total_count:
        return 'Texto'
    else:
        return 'Categórico'

def categories(df):
    return {c: category(df[c]) for c in df.columns}

Tipos de dados no momento

In [6]:
categories(df)

{'cargo': 'Categórico',
 'codigo_do_orgao_superior': 'Numérico',
 'codigo_orgao_solicitante': 'Numérico',
 'cpf_viajante': 'Categórico',
 'descricao_funcao': 'Categórico',
 'destinos': 'Categórico',
 'funcao': 'Categórico',
 'identificador_do_processo_de_viagem': 'Numérico',
 'justificativa_urgencia_viagem': 'Categórico',
 'motivo': 'Categórico',
 'nome': 'Categórico',
 'nome_do_orgao_superior': 'Categórico',
 'nome_orgao_solicitante': 'Categórico',
 'numero_da_proposta_pcdp': 'Categórico',
 'periodo_data_de_fim': 'Categórico',
 'periodo_data_de_inicio': 'Categórico',
 'situacao': 'Categórico',
 'valor_diarias': 'Categórico',
 'valor_outros_gastos': 'Categórico',
 'valor_passagens': 'Categórico',
 'viagem_urgente': 'Categórico'}

Função para normalizar dados textuais

In [7]:
import unicodedata

def normalize(string):
    return unicodedata.normalize("NFKD", str(string)).encode('ascii', errors='ignore').decode('utf-8')

Análise da coluna Identificador do processo de viagem: dados inteiros já normalizados

In [8]:
df.identificador_do_processo_de_viagem.sample(5)

47324     17335551
40824     17326230
55485     17346999
135354    17468359
64672     17359644
Name: identificador_do_processo_de_viagem, dtype: int64

Análise da coluna número da proposta PCDP: dados textuais a serem normalizados 

In [9]:
df.numero_da_proposta_pcdp.sample(5)

1088           000113/21
5919     Informações pro
25912    Informações pro
46403    Informações pro
78785          001399/21
Name: numero_da_proposta_pcdp, dtype: object

In [10]:
df.numero_da_proposta_pcdp = df.numero_da_proposta_pcdp.apply(normalize)

Análise da coluna situação: dados textuais a serem normalizados

In [11]:
df.situacao.sample(5)

67147         Realizada
3157          Realizada
100657        Realizada
46234     Não realizada
52675         Realizada
Name: situacao, dtype: object

In [12]:
df.situacao = df.situacao.apply(normalize)

Análise da coluna viagem urgente: dados textuais a serem normalizados

In [13]:
df.viagem_urgente.sample(5)

53352    NÃO
37875    SIM
61545    NÃO
85420    NÃO
49932    SIM
Name: viagem_urgente, dtype: object

In [14]:
df.viagem_urgente = df.viagem_urgente.apply(normalize)

Análise da coluna justificativa de urgência de viagem: dados textuais a serem normalizados

In [15]:
df.justificativa_urgencia_viagem.sample(5)

25838     Informação protegida por sigilo nos termos da ...
75884     Informação protegida por sigilo nos termos da ...
20344     Informação protegida por sigilo nos termos da ...
131007                                       Sem informação
100465    Informação protegida por sigilo nos termos da ...
Name: justificativa_urgencia_viagem, dtype: object

In [16]:
df.justificativa_urgencia_viagem = df.justificativa_urgencia_viagem.apply(normalize)

Análise da coluna código do orgão superior: dados inteiros já normalizados

In [17]:
df.codigo_do_orgao_superior.sample(5) 

28466     30000
146899    20000
2150      52000
142954    39000
65798     26000
Name: codigo_do_orgao_superior, dtype: int64

Análise da coluna nome do orgão superior: dados textuais a serem normalizados

In [18]:
df.nome_do_orgao_superior.sample(5)

110999    Ministério da Agricultura, Pecuária e Abasteci...
72869                                  Ministério da Defesa
40840             Ministério da Justiça e Segurança Pública
116961                               Ministério da Educação
21851                                   Ministério da Saúde
Name: nome_do_orgao_superior, dtype: object

In [19]:
df.nome_do_orgao_superior = df.nome_do_orgao_superior.apply(normalize)

Análise da coluna código do orgão solicitante: dados inteiros já normalizados

In [20]:
df.codigo_orgao_solicitante.sample(5)

19150     52121
79812     20000
58512        -1
104425    39000
41911     20000
Name: codigo_orgao_solicitante, dtype: int64

Análise da coluna nome do orgão solicitante: dados textuais a serem normalizados

In [21]:
df.nome_orgao_solicitante.sample(5)

46816                     Comando do Exército
135521        Departamento de Polícia Federal
133548                    Comando do Exército
138577                         Sem informação
99998     Companhia Nacional de Abastecimento
Name: nome_orgao_solicitante, dtype: object

In [22]:
df.nome_orgao_solicitante = df.nome_orgao_solicitante.apply(normalize)

Análise da coluna CPF do viajante: dados textuais a serem normalizados

In [23]:
df.cpf_viajante.sample(5)

133785       ID005500312
89898        ID002500217
1626      ***.702.064-**
125323       ID005800236
90382     ***.589.299-**
Name: cpf_viajante, dtype: object

In [24]:
df.cpf_viajante = df.cpf_viajante.apply(normalize)

Análise da coluna nome: dados textuais a serem normalizados

In [25]:
df.nome.sample(5)

68899                      WALLEX SANTOS DE LIMA
58931     CARLOS ADRIANO FERREIRA BENTES RIBEIRO
51012                        CARLA MIOTO NICIANI
147293              JESSICA HELLEN CARDOSO LOPES
79978                  CASSIANO PEDRO DOS SANTOS
Name: nome, dtype: object

In [26]:
df.nome = df.nome.apply(normalize)

Análise da coluna cargo: dados textuais a serem normalizados

In [27]:
df.cargo.sample(5)

130370    Informações protegidas por sigilo
26386                                   NaN
82950                AUXILIAR DE SANEAMENTO
107528             TECNICO EM ELETROTECNICA
50845     Informações protegidas por sigilo
Name: cargo, dtype: object

In [28]:
df.cargo = df.cargo.apply(normalize)

Análise da coluna função: dados textuais a serem normalizados

In [29]:
df.funcao.sample(5)

131308            -1
87111             -1
113139       CD-0001
144968      DAS-1021
147029    OfSuperior
Name: funcao, dtype: object

In [30]:
df.funcao = df.funcao.apply(normalize)

Análise da coluna descrição da função: dados textuais a serem normalizados

In [31]:
df.descricao_funcao.sample(5)

131045    Informações protegidas por sigilo
68130                            Suboficial
47720                         Não Informado
130882    Informações protegidas por sigilo
133469        Cargo Comissionado de Direção
Name: descricao_funcao, dtype: object

In [32]:
df.descricao_funcao = df.descricao_funcao.apply(normalize)

Análise da coluna período da data de inicio: dados que precisam ser convertidos para formato data

In [33]:
df.periodo_data_de_inicio.head(5)

0    03/02/2021
1    11/05/2021
2    23/02/2021
3    24/04/2021
4    04/01/2021
Name: periodo_data_de_inicio, dtype: object

In [34]:
df.periodo_data_de_inicio = pd.to_datetime(df.periodo_data_de_inicio, format="%d/%m/%Y")

Análise da coluna período da data de fim: dados que precisam ser convertidos para data 

In [35]:
df.periodo_data_de_fim.head(5)

0    04/02/2021
1    16/05/2021
2    23/02/2021
3    28/04/2021
4    04/01/2021
Name: periodo_data_de_fim, dtype: object

In [36]:
df.periodo_data_de_fim = pd.to_datetime(df.periodo_data_de_fim, format="%d/%m/%Y")

Análise da coluna destino: dados textuais a serem normalizados

In [37]:
df.destinos.sample(5)

138797               Queluz/SP, Miracatu/SP
34124                      Florianópolis/SC
88004                            Caracol/MS
65981                             Itajaí/SC
98381     Informações protegidas por sigilo
Name: destinos, dtype: object

In [38]:
df.destinos = df.destinos.apply(normalize)

Análise da coluna motivo: dados textuais a serem normalizados

In [39]:
df.motivo.sample(5)

40144     Participação em reunião do NUREP (Núcleo Regio...
124684    OS n. 2165/GUARNAE-CT/2021 - PARTICIPAR COMO P...
1410      Informação protegida por sigilo nos termos da ...
137711    Participar de Visita ao entreposto de Contagem...
59462     Informação protegida por sigilo nos termos da ...
Name: motivo, dtype: object

In [40]:
df.motivo = df.motivo.apply(normalize)

Análise da coluna valor diárias: dados que precisam ser convertidos para float

In [41]:
df.valor_diarias.head(5)

0    223,86
1      0,00
2      0,00
3      0,00
4      0,00
Name: valor_diarias, dtype: object

In [42]:
def convert_brazil_string_to_float(x):
  return float(x.replace(",", "."))

df.valor_diarias = df.valor_diarias.apply(convert_brazil_string_to_float)

Análise da coluna valor de passagens: dados que precisam ser convertidos para float

In [43]:
df.valor_passagens.sample(5)

127719       0,00
88529        0,00
10938     3995,73
136723       0,00
1405      1753,10
Name: valor_passagens, dtype: object

In [44]:
df.valor_passagens = df.valor_passagens.apply(convert_brazil_string_to_float)

Análise da coluna valor outros gastos: dados que precisam ser convertidos para float

In [45]:
df.valor_outros_gastos.sample(5)

146638    0,00
105423    0,00
54727     0,00
101138    0,00
91354     0,00
Name: valor_outros_gastos, dtype: object

In [46]:
df.valor_outros_gastos = df.valor_outros_gastos.apply(convert_brazil_string_to_float)

Categorias após tratamento das colunas

In [47]:
categories(df)

{'cargo': 'Categórico',
 'codigo_do_orgao_superior': 'Numérico',
 'codigo_orgao_solicitante': 'Numérico',
 'cpf_viajante': 'Categórico',
 'descricao_funcao': 'Categórico',
 'destinos': 'Categórico',
 'funcao': 'Categórico',
 'identificador_do_processo_de_viagem': 'Numérico',
 'justificativa_urgencia_viagem': 'Categórico',
 'motivo': 'Categórico',
 'nome': 'Categórico',
 'nome_do_orgao_superior': 'Categórico',
 'nome_orgao_solicitante': 'Categórico',
 'numero_da_proposta_pcdp': 'Categórico',
 'periodo_data_de_fim': 'Data',
 'periodo_data_de_inicio': 'Data',
 'situacao': 'Categórico',
 'valor_diarias': 'Numérico',
 'valor_outros_gastos': 'Numérico',
 'valor_passagens': 'Numérico',
 'viagem_urgente': 'Categórico'}

In [48]:
df.to_pickle("2021_Viagem_tratado.pkl")

# Expressões regulares

Escreva as expressões regulares solicitadas. Cada exercício possui um conjunto de testes que deve ser completamente validado pela expressão regular. Cada teste possui registros onde expera-se um *match* da expressão regular, enquanto outros registros não devem apresentar nenhum *match*. Faça o exercício para que todos os registros do teste fiquem com a marca de correto (em verde). Para os exercícios, não utilize expressões regulares com mais de 80 caracteres.



Para esse exercício você precisará baixar o arquivo zipado "exerciciosRegex" disponibilizado na pasta QADS em (https://drive.google.com/drive/folders/1sUeC2VYx93ZXi3tKCJarUTVvq_7C2ECI?usp=sharing), descompactar esse arquivo e copiar todo o conteúdo em uma pasta do seu Google drive (contém um diretório chamado "exercicios" e um arquivo chamado re_jupyter.py). Em seguida você deve montar seu Google Drive aqui no Colab e acessar a pasta onde copiou esse conteúdo .

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
%cd /content/drive/MyDrive/'Academia QADS'/'Obtendo e Preparando dados'/exerciciosRegex/

/content/drive/MyDrive/Academia QADS/Obtendo e Preparando dados/exerciciosRegex


In [51]:
%ls

exercicios/  __pycache__/  re_jupyter.py


In [52]:
import imp
import re_jupyter
imp.reload(re_jupyter)

from re_jupyter import exercicio

Uma letra `a` imediatamente seguida por uma ou mais letras `b`

In [53]:
exercicio(1, r'(ab+)')

,ab
,abb
,abbbbbbbbb
,cbaaabb
,aba
,a (sem ocorrências)
,baa (sem ocorrências)
,ba (sem ocorrências)
,cba (sem ocorrências)
,cbaaa (sem ocorrências)
,ba (sem ocorrências)


Uma palavra de 6 letras que inicia com a letra `z`

In [54]:
exercicio(2, r'\bz\w{5}\b')

,zangas
,zangas espaço
,zéfiro
,segundo zéfiro
,zelo (sem ocorrências)
,zelos (sem ocorrências)
,zangada (sem ocorrências)
,zangados (sem ocorrências)
,azul (sem ocorrências)
,aziago (sem ocorrências)
,azougue (sem ocorrências)


Uma palavra com 8 a 10 letras que termine com a letra 'z'

In [55]:
exercicio(3, r'\b\w{7,9}z\b')

,robustez
,espaço robustez
,chafariz
,antes chafariz depois
,vetustez
,placidez
,insensatez
,gravidez
,invalidez
,luz (sem ocorrências)
,talvez (sem ocorrências)


3 dígitos consecutivos

In [56]:
exercicio(4, r'\d{3}')

,040 inicia com zero
,056 de novo
,100 cem
,789 789 789
,0101 binário
,000x termina em x
,ano de 1984
,data 01/01/2018
,zero 0 (sem ocorrências)
,dígito 1 (sem ocorrências)
,vinte 20 (sem ocorrências)


Um número inteiro com 2, 3 ou 4 dígitos, sendo que o primeiro dígito não pode ser zero

In [57]:
exercicio(5, r'\b[1-9]\d{1,3}\b')

,vinte 20
,100 cem
,789 789 789
,ano de 1984
,data 01/01/2018
,zero 0 (sem ocorrências)
,dígito 1 (sem ocorrências)
,inicia com zero 03 (sem ocorrências)
,03 também tem zero (sem ocorrências)
,040 inicia com zero (sem ocorrências)
,056 de novo (sem ocorrências)


Uma palavra de mais de dois caracteres que inicie com um dígito e termine com uma letra

In [111]:
exercicio(6, r'\b\d[\d\w]+[^\d\s]\b')

,parágrafo 11b
,1111111b
,1bbbbbbb
,1b1b1b1b
,teste 1a (sem ocorrências)
,dígitos 113 total (sem ocorrências)
,soma quando 3a (sem ocorrências)
,bb1bbbbb11b (sem ocorrências)
,b1b1b1b1 (sem ocorrências)
,111 (sem ocorrências)
,1a1 (sem ocorrências)


Uma palavra seguida de um espaço seguido de um sequência de um ou mais dígitos

In [59]:
exercicio(7, r'\w+\s\d+')

,teste 1234
,parágrafo 12345
,letra 1
,a b 1 c
,cc 11 a
,1 1
,dois espaços não pode 2 (sem ocorrências)


Um dia da semana: segunda-feira, terça-feira, quarta-feira, quinta-feira, sexta-feira. Adicionalmente, permita a escrita dos dias da semana com espaço no lugar do hífen ou com as palavras unidas (ex.: quartafeira e quinta feira).

In [60]:
exercicio(8, r'segunda[-\s]?feira|\bterça[-\s]?feira|quarta[-\s]?feira|quinta[-\s]?feira|^sexta[-\s]?feira')

,hoje é segunda-feira
,amanhã será terça feira
,quartafeira já passou
,quinta-feira foi dia 3
,sextafeira
,sábado feriado (sem ocorrências)
,domingo feriado (sem ocorrências)
,2a feira internacional (sem ocorrências)
,sexta_feira (sem ocorrências)
,quinta.feira (sem ocorrências)
,domingues (sem ocorrências)


Uma URL no formato: http://www.&lt;dominio&gt;.com.br/&lt;arquivo&gt;.html. O &lt;arquivo&gt; e o &lt;domínio&gt; somente podem contar letras de a a z (minúscula ou maiúsculas), dígitos e os sinais _ e -.

In [61]:
exercicio(9, r'http://www\.[\w\d_-]+.com\.br/[\w\d_-]+\.html')

,http://www.google.com.br/index.html
,http://www.terra.com.br/underline_1.html
,http://www.teste-novo.com.br/hiFen-1.html
,http://www.google.com/index.html (sem ocorrências)
,http://uol.com.br/teste.html (sem ocorrências)
,http://bol.com.br/nao/podem/existir/barras.html (sem ocorrências)
,https://www.google.com/https/também/nao/pode.html (sem ocorrências)
,http://www.teste_velha.com.br/111-html (sem ocorrências)
,http://www-testevelho.com-br/111.html (sem ocorrências)
,http://www..com-br/1.html (sem ocorrências)
,http://www.com-br/2.html (sem ocorrências)


Duas palavras unidas pelas preposições 'de', 'do', 'da', 'dos' ou 'das'. As palavras somente podem conter letras e elas são separadas por um ou mais espaços.

In [62]:
exercicio(10, r'\w+\s+d[eoa]s?\s+\w+')

,"Olhos do mundo,"
,"zelos do marido,"
,"morte do Viegas,"
,uma vertigem de cabriolas
,descendente do cavalo de Aquiles
,a minha teoria das edições humanas
,me pareceu entrar na região dos gelos eternos
,"Esta idéia, (sem ocorrências)"
,"rútila e grande, (sem ocorrências)"
,"trajada ao bizarro, (sem ocorrências)"
,"como diria o Padre Bernardes, (sem ocorrências)"


Uma palavra de no mínimo 3 letras e com TODAS as letras maiúsculas

In [63]:
exercicio(11, r'\b[A-ZÓÉ]{3,}\b')

,"ALBERTO YOUSSEF, brasileiro, casado, filho de Kalim Youssef e Antonieta Youssef"
,"PAULO ROBERTO COSTA, ex-Diretor de Abastecimento da PETROBRAS"
,"WALDOMIRO DE OLIVEIRA, conhecido “Bom Velhinho”"
,"JOSÉ ALDEMÁRIO PINHEIRO FILHO, vulgo “Léo Pinheiro”,"
,"AGENOR FRANKLIN MAGALHÃES MEDEIROS, brasileiro"
,empreiteiras ODEBRECHT
,integrantes do Grupo OAS
,empresa GFD INVESTIMENTOS
,conforme já apurado pelo TCU
,"no caso das empresas MO Consultoria, Empreiteira Rigidez e RCI Software"
,RELATÓRIO DE TENDÊNCIAs


Duas palavras consecutivas, de no mínimo 2 letras cada, com TODAS as letras maiúsculas

In [64]:
exercicio(12, r'\b[A-ZÁÓÉÊ]{2,}\s+[A-ZÁÓÉÊ]{2,}\b')

,"ALBERTO YOUSSEF, brasileiro, casado, filho de Kalim Youssef e Antonieta Youssef"
,"PAULO ROBERTO COSTA, ex-Diretor de Abastecimento da PETROBRAS"
,"WALDOMIRO DE OLIVEIRA, conhecido “Bom Velhinho”"
,"JOSÉ ALDEMÁRIO PINHEIRO FILHO, vulgo “Léo Pinheiro”,"
,"AGENOR FRANKLIN MAGALHÃES MEDEIROS, brasileiro"
,empresa GFD INVESTIMENTOS
,RELATÓRIO DE TENDÊNCIAs
,VARIOS ESPACOS
,empreiteiras ODEBRECHT (sem ocorrências)
,integrantes do Grupo OAS (sem ocorrências)
,conforme já apurado pelo TCU (sem ocorrências)


Duas ou mais palavras consecutivas com todas as letras maiúsculas. Cada palavra pode possuir qualquer quantidade de letras.

In [65]:
exercicio(13, r'\b[A-ZÓÉ]+\s+[A-ZÁ]+\b(\s[A-ZÃÊ]+)?(\s[A-Z]+)?\b')

,"ALBERTO YOUSSEF, brasileiro, casado, filho de Kalim Youssef e Antonieta Youssef"
,"PAULO ROBERTO COSTA, ex-Diretor de Abastecimento da PETROBRAS"
,"WALDOMIRO DE OLIVEIRA, conhecido “Bom Velhinho”"
,"JOSÉ ALDEMÁRIO PINHEIRO FILHO, vulgo “Léo Pinheiro”,"
,"AGENOR FRANKLIN MAGALHÃES MEDEIROS, brasileiro"
,empresa GFD INVESTIMENTOS
,RELATÓRIO DE TENDÊNCIAs
,TESTE A TESTE
,A BE C DE
,VARIOS ESPACOS
,empreiteiras ODEBRECHT (sem ocorrências)


Uma linha contendo exatamente um endereço [IP](https://en.wikipedia.org/wiki/IP_address). ex.: 127.0.0.1, 192.168.0.254. Um IP é formado por exatamente 4 números separados por pontos. Cada número possui no máximo 3 dígitos.

In [66]:
exercicio(14, r'^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$')

,127.0.0.1
,192.168.0.254
,10.11.10.11
,127.127.127.127
,9999.0.0.1 (sem ocorrências)
,127404041 (sem ocorrências)
,a127.0.0.1 (sem ocorrências)
,127.a.a.1 (sem ocorrências)
,127.0.0.1a (sem ocorrências)
,xxx.yyy.zzz.www (sem ocorrências)
,0.0.0.a (sem ocorrências)


Uma linha contendo exatamente um CPF com separador de milhares e hífen.

In [67]:
exercicio(15, r'^\d{3}\.\d{3}\.\d{3}\-\d{2}$')

,000.000.000-00
,111.111.111-11
,0111.111.111-11 (sem ocorrências)
,111.111.111-110 (sem ocorrências)
,111.0111.0111-11 (sem ocorrências)
,11101110111-11 (sem ocorrências)
,111.111.111111 (sem ocorrências)
,1110110111111 (sem ocorrências)
,111011011111 (sem ocorrências)
,11101101111 (sem ocorrências)
,1110110111 (sem ocorrências)


Uma linha contendo exatamente um CPF com separador de milhares opcional e hífen opcional.

In [68]:
exercicio(16, r'^\d{3}\.?\d{3}\.?\d{3}\-?\d{2}$')

,000.000.000-00
,111.111.111-11
,111111.111-11
,111.111111-11
,11101101111
,0111.111.111-11 (sem ocorrências)
,111.111.111-110 (sem ocorrências)
,111.0111.0111-11 (sem ocorrências)
,11101110111-11 (sem ocorrências)
,111.111.111111 (sem ocorrências)
,1110110111111 (sem ocorrências)


Uma linha contendo exatamente um CNPJ com separador de milhares, barra e hífen.

In [69]:
exercicio(17, r'^\d{2}\.\d{3}\.\d{3}\/\d{4}\-\d{2}$')

,00.000.000/0001-00
,xx.000.000/0001-xx (sem ocorrências)
,0011001000/0001-00 (sem ocorrências)
,00.000.00010001-00 (sem ocorrências)
,00.000.000/0001100 (sem ocorrências)
,00000.000/0001-00 (sem ocorrências)
,00000000/0001-00 (sem ocorrências)
,000000000001-00 (sem ocorrências)
,00000000000100 (sem ocorrências)
,x00000000000100 (sem ocorrências)
,00000000000100x (sem ocorrências)


Uma linha contendo exatamente um CNPJ com separador de milhares opcional, barra opcional e hífen opcional.

In [70]:
exercicio(18, r'^\d{2}\.?\d{3}\.?\d{3}\/?\d{4}\-?\d{2}$')

,00.000.000/0001-00
,00000.000/0001-00
,00000000/0001-00
,000000000001-00
,00000000000100
,xx.000.000/0001-xx (sem ocorrências)
,0011001000/0001-00 (sem ocorrências)
,00.000.00010001-00 (sem ocorrências)
,00.000.000/0001100 (sem ocorrências)
,x00000000000100 (sem ocorrências)
,00000000000100x (sem ocorrências)


Uma linha contendo exatamente um CEP com separador de milhares e hífen

In [71]:
exercicio(19, r'^\d{2}\.\d{3}\-\d{3}')

,70.000-000
,7a.aaa-aaa (sem ocorrências)
,7000000000 (sem ocorrências)
,70000-000 (sem ocorrências)
,700000000 (sem ocorrências)
,70000000 (sem ocorrências)
,7000000 (sem ocorrências)
,700000 (sem ocorrências)
,70000 (sem ocorrências)
,7000 (sem ocorrências)
,70000aaa (sem ocorrências)


Uma linha contendo exatamente um CEP com separador de milhares opcional e hífen opcional

In [72]:
exercicio(20, r'^\d{2}\.?\d{3}\-?\d{3}$')

,70.000-000
,70000-000
,70000000
,7a.aaa-aaa (sem ocorrências)
,7000000000 (sem ocorrências)
,700000000 (sem ocorrências)
,7000000 (sem ocorrências)
,700000 (sem ocorrências)
,70000 (sem ocorrências)
,7000 (sem ocorrências)
,70000aaa (sem ocorrências)


Uma linha contendo exatamente um telefone no formato (xx)xxxx-xxxx.

In [73]:
exercicio(21, r'^\(\d{2}\)\d{4}\-\d{4}$')

,(61)3316-0000
,3316-0000 (sem ocorrências)
,x61x3316x0000 (sem ocorrências)
,(61)3316x0000 (sem ocorrências)
,((61)3316-00006 (sem ocorrências)
,77(61)3316x000077 (sem ocorrências)
,(77)(61)3316-00007 (sem ocorrências)


Uma linha contendo exatamente um telefone no formato (xx)xxxx-xxxx ou xxxx-xxxx (sem DDD).

In [74]:
exercicio(22, r'^\(\d{2}\)\d{4}\-\d{4}|^\d{4}\-\d{4}')

,(61)3316-0000
,3316-0000
,x61x3316x0000 (sem ocorrências)
,(61)3316x0000 (sem ocorrências)
,((61)3316-00006 (sem ocorrências)
,77(61)3316x000077 (sem ocorrências)
,(77)(61)3316-00007 (sem ocorrências)


# JSON

Usando a API do Banco Central, descrita em https://www.bcb.gov.br/conteudo/dadosabertos/BCBDepin/gnastportal-dados-abertostaxas-de-cambio---todos-os-boletins-diarios.pdf, escreva uma função que apresente ao usuário as moedas disponíveis para cotação, peça a ele que escolha uma ou mais, peça que o usuário escolha uma data inicial e uma data final, obtenha as cotações para as moedas selecionadas no período, armazene num dataframe e ao final salve o dataframe no disco usando o Pickle.

# HTML, JSON e REGEX

Crie um extrator de contratos a partir da seção 3 do Diário Oficial da União, obtendo as informações da página (link e texto) e por meio de expressões regulares extraia o nome do contratante, CNPJ e nome do contratado. Coloque o resultado num dataframe e salve em disco com o Pickle.